In [1]:
import pandas as pd 
import numpy as np
import warnings

warnings.filterwarnings(action='ignore')

### 데이터 불러오기 

In [2]:
cust_tr = pd.read_csv('cust_train.csv', encoding = 'utf-8') #회원
product_tr = pd.read_csv('product_train.csv',encoding = 'utf-8') #상품구매
search_tr = pd.read_csv('search_train.csv',encoding = 'utf-8') #검색어
session_tr = pd.read_csv('session_train.csv',encoding = 'utf-8') #세션

In [24]:
cust_te = pd.read_csv('sample_submission.csv', encoding = 'utf-8')
product_te = pd.read_csv('product_test.csv',encoding = 'utf-8')
search_te = pd.read_csv('search_test.csv',encoding = 'utf-8')
session_te = pd.read_csv('session_test.csv',encoding = 'utf-8')

In [25]:
master = pd.read_csv('master.csv', encoding = 'utf-8') #상품분류

### 데이터 전처리

In [26]:
train = pd.merge(product_tr, session_tr, on = ['CLNT_ID','SESS_ID'])
train = pd.merge(train, search_tr, on = ['CLNT_ID','SESS_ID'])
train = pd.merge(train, master, on = 'PD_C')

In [27]:
test = pd.merge(product_te, session_te, on = ['CLNT_ID','SESS_ID'])
test = pd.merge(test, search_te, on = ['CLNT_ID','SESS_ID'])
test = pd.merge(test, master, on = 'PD_C')

In [28]:
train['PD_BUY_AM']  = train['PD_BUY_AM'].map(lambda x: int(str(x).replace(',','')) )
train['PD_BUY_CT']  = train['PD_BUY_CT'].map(lambda x: int(str(x).replace(',','')) )
test['PD_BUY_AM']  = test['PD_BUY_AM'].map(lambda x: int(str(x).replace(',','')) )
test['PD_BUY_CT']  = test['PD_BUY_CT'].map(lambda x: int(str(x).replace(',','')) )

- 상품 1개당 금액과 구매수량을 int 타입으로 바꿈

### feature 생성

In [30]:
features = []
features_te = []

**[총구매액, 구매건수, 평균구매액, 최대구매액]**

In [31]:
train['AMOUNT'] = train['PD_BUY_AM'] * train['PD_BUY_CT']
test['AMOUNT'] = test['PD_BUY_AM'] * test['PD_BUY_CT']

In [32]:
train

,CLNT_ID,SESS_ID,HITS_SEQ,PD_C,PD_ADD_NM,PD_BRA_NM,PD_BUY_AM,PD_BUY_CT,SESS_SEQ,SESS_DT,...,DVC_CTG_NM,ZON_NM,CITY_NM,KWD_NM,SEARCH_CNT,PD_NM,CLAC1_NM,CLAC2_NM,CLAC3_NM,AMOUNT
0,128408,8274151,356,283756,모델:선택02.CQ2332|사이즈:260 / 1개,아디다스(퍼포먼스),42500,1,19,20180515,...,mobile,Seoul,Seoul,여아신발,1,니짜 6종 택1 (CQ2333 CQ2332 CQ2331 CQ2537 CQ3104 C...,스포츠패션,여성스포츠화,여성런닝/트레이닝화,42500
1,128408,8274151,356,283756,모델:선택02.CQ2332|사이즈:260 / 1개,아디다스(퍼포먼스),42500,1,19,20180515,...,mobile,Seoul,Seoul,아디다스키즈,1,니짜 6종 택1 (CQ2333 CQ2332 CQ2331 CQ2537 CQ3104 C...,스포츠패션,여성스포츠화,여성런닝/트레이닝화,42500
2,128408,8274151,356,283756,모델:선택02.CQ2332|사이즈:260 / 1개,아디다스(퍼포먼스),42500,1,19,20180515,...,mobile,Seoul,Seoul,아디다스 바람막이,1,니짜 6종 택1 (CQ2333 CQ2332 CQ2331 CQ2537 CQ3104 C...,스포츠패션,여성스포츠화,여성런닝/트레이닝화,42500
3,128408,8274151,356,283756,모델:선택02.CQ2332|사이즈:260 / 1개,아디다스(퍼포먼스),42500,1,19,20180515,...,mobile,Seoul,Seoul,신발,1,니짜 6종 택1 (CQ2333 CQ2332 CQ2331 CQ2537 CQ3104 C...,스포츠패션,여성스포츠화,여성런닝/트레이닝화,42500
4,128408,8274151,356,283756,모델:선택02.CQ2332|사이즈:240 / 1개,아디다스(퍼포먼스),42500,1,19,20180515,...,mobile,Seoul,Seoul,여아신발,1,니짜 6종 택1 (CQ2333 CQ2332 CQ2331 CQ2537 CQ3104 C...,스포츠패션,여성스포츠화,여성런닝/트레이닝화,42500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3415958,175360,857913,56,25522,색상:검정색|Size:S / 1개,에잇세컨즈,39900,1,46,20180916,...,mobile,Seoul,Seoul,에잇세컨즈,1,여성 [LAB8] 블랙 코튼 오버사이즈 후드 티셔츠 (358941CYB5) - 검정...,남성의류,남성의류상의,남성티셔츠,39900
3415959,175360,857913,56,25522,색상:검정색|Size:S / 1개,에잇세컨즈,39900,1,46,20180916,...,mobile,Seoul,Seoul,쿠폰,1,여성 [LAB8] 블랙 코튼 오버사이즈 후드 티셔츠 (358941CYB5) - 검정...,남성의류,남성의류상의,남성티셔츠,39900
3415960,180188,10554141,109,491849,색상:네이비|사이즈:66 / 1개,베스띠벨리,39000,1,5,20180407,...,mobile,Chungcheongnam-do,Asan-si,올리비아로렌,1,넥포인트 블라우스 BQB2486 - 네이비 / 55,여성의류,여성의류상의,여성남방셔츠,39000
3415961,180188,10554141,109,394319,색상:그린(GN)|사이즈:66 / 1개,쉬즈미스,39000,1,5,20180407,...,mobile,Chungcheongnam-do,Asan-si,올리비아로렌,1,스트라이프 롤업셔츠(SWWSTH11020) - 그린(GN) / 66,여성의류,여성의류상의,여성남방셔츠,39000


- 한 사람이 구매한 총 금액을  알기 위해 추가

In [33]:
train.AMOUNT

0          42500
1          42500
2          42500
3          42500
4          42500
           ...  
3415958    39900
3415959    39900
3415960    39000
3415961    39000
3415962    39900
Name: AMOUNT, Length: 3415963, dtype: int64

In [34]:
f = train.groupby('CLNT_ID')['AMOUNT'].agg([('총구매액', np.sum),
                                            ('구매건수', np.size),
                                            ('평균구매액', lambda x : np.round(np.mean(x))),
                                            ('최대구매액', np.max)]).reset_index()
features.append(f);display(f)

f_te = test.groupby('CLNT_ID')['AMOUNT'].agg([('총구매액', np.sum),
                                            ('구매건수', np.size),
                                            ('평균구매액', lambda x : np.round(np.mean(x))),
                                            ('최대구매액', np.max)]).reset_index()
features_te.append(f_te) ; display(f_te)

,CLNT_ID,총구매액,구매건수,평균구매액,최대구매액
0,0,86500,2,43250,81000
1,1,1276000,16,79750,99000
2,2,560000,11,50909,62100
3,3,851200,8,106400,110400
4,4,125200,6,20867,37000
...,...,...,...,...,...
263099,263099,973000,112,8688,22000
263100,263100,22400,2,11200,12900
263101,263101,135600,4,33900,33900
263102,263102,59400,15,3960,9720


,CLNT_ID,총구매액,구매건수,평균구매액,최대구매액
0,263104,31600,3,10533,13800
1,263105,314000,4,78500,120000
2,263106,49800,2,24900,29900
3,263107,88000,2,44000,49000
4,263108,124000,1,124000,124000
...,...,...,...,...,...
112755,375859,80000,3,26667,35000
112756,375860,902800,12,75233,149000
112757,375861,91800,2,45900,45900
112758,375862,92000,4,23000,23000


**[주말방문비율]**

In [35]:
train['date'] = pd.to_datetime(train['SESS_DT'], format= '%Y%m%d')

In [36]:
test['date'] = pd.to_datetime(test['SESS_DT'], format= '%Y%m%d')

- 구매일자를 datetime 타입으로 변형

In [37]:
f = train.groupby('CLNT_ID')['date'].agg([
    ('주말방문비율', lambda x: np.mean(x.dt.dayofweek>4))]).reset_index()
features.append(f); display(f)

f_te = test.groupby('CLNT_ID')['date'].agg([
    ('주말방문비율', lambda x: np.mean(x.dt.dayofweek>4))]).reset_index()
features_te.append(f_te);display(f_te)

,CLNT_ID,주말방문비율
0,0,1.000000
1,1,0.125000
2,2,0.000000
3,3,1.000000
4,4,0.333333
...,...,...
263099,263099,0.000000
263100,263100,0.500000
263101,263101,0.000000
263102,263102,0.000000


,CLNT_ID,주말방문비율
0,263104,0.00
1,263105,0.50
2,263106,1.00
3,263107,0.00
4,263108,1.00
...,...,...
112755,375859,0.00
112756,375860,0.25
112757,375861,0.00
112758,375862,0.00


**[계절방문비율]**

In [38]:
f = train.groupby('CLNT_ID')['date'].agg([
    ('봄-구매비율', lambda x: np.mean( x.dt.month.isin([3,4,5]))),
    ('여름-구매비율', lambda x: np.mean( x.dt.month.isin([6,7,8]))),
    ('가을-구매비율', lambda x: np.mean(x.dt.month.isin([9,10,11]))),
    ('겨울-구매비율', lambda x: np.mean( x.dt.month.isin([1,2,12])))
]).reset_index()
features.append(f); f

,CLNT_ID,봄-구매비율,여름-구매비율,가을-구매비율,겨울-구매비율
0,0,0.000000,1.000000,0.000000,0.0
1,1,0.375000,0.625000,0.000000,0.0
2,2,0.000000,0.909091,0.090909,0.0
3,3,0.000000,0.000000,1.000000,0.0
4,4,0.666667,0.333333,0.000000,0.0
...,...,...,...,...,...
263099,263099,0.000000,1.000000,0.000000,0.0
263100,263100,0.500000,0.500000,0.000000,0.0
263101,263101,1.000000,0.000000,0.000000,0.0
263102,263102,1.000000,0.000000,0.000000,0.0


In [39]:
f_te = test.groupby('CLNT_ID')['date'].agg([
    ('봄-구매비율', lambda x: np.mean( x.dt.month.isin([3,4,5]))),
    ('여름-구매비율', lambda x: np.mean( x.dt.month.isin([6,7,8]))),
    ('가을-구매비율', lambda x: np.mean(x.dt.month.isin([9,10,11]))),
    ('겨울-구매비율', lambda x: np.mean( x.dt.month.isin([1,2,12])))
]).reset_index()
features_te.append(f_te); f_te

,CLNT_ID,봄-구매비율,여름-구매비율,가을-구매비율,겨울-구매비율
0,263104,0.000000,1.000000,0.0,0.0
1,263105,0.000000,1.000000,0.0,0.0
2,263106,0.000000,1.000000,0.0,0.0
3,263107,1.000000,0.000000,0.0,0.0
4,263108,0.000000,1.000000,0.0,0.0
...,...,...,...,...,...
112755,375859,1.000000,0.000000,0.0,0.0
112756,375860,0.583333,0.416667,0.0,0.0
112757,375861,0.000000,1.000000,0.0,0.0
112758,375862,0.000000,1.000000,0.0,0.0


### Merge features

In [40]:
data = pd.DataFrame({'CLNT_ID': train.CLNT_ID.unique()})
for f in features :
    data = pd.merge(data, f, how='left')
    
data = data.fillna(0)

In [41]:
data_te = pd.DataFrame({'CLNT_ID': test.CLNT_ID.unique()})
for f in features_te :
    data_te = pd.merge(data_te, f, how='left')
    
data_te = data_te.fillna(0)

### Model 생성

In [42]:
from sklearn.linear_model import LogisticRegression

# 모델 생성 및 학습
lr = LogisticRegression()
lr.fit(data, cust_tr.LABEL)

# 테스트 데이터 예측
pred = pd.DataFrame(lr.predict_proba(data_te))

# 결과값 정제 및 내보내기
result = pd.concat([cust_te.CLNT_ID, pred], axis=1)
result.columns = ['CLNT_ID','F20','F30','F40','M20','M30','M40']
result.to_csv('조이름(1).csv',index=False)